## Imports

In [19]:
from keras.models import load_model
from os import listdir
from PIL import Image
from numpy import savez_compressed
from numpy import asarray, load, expand_dims
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot
from mtcnn import MTCNN
import pickle

In [2]:
model = load_model('Serialized/facenet_keras.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
print(model.inputs)
print(model.outputs)

[<tf.Tensor 'input_1:0' shape=(?, 160, 160, 3) dtype=float32>]
[<tf.Tensor 'Bottleneck_BatchNorm/cond/Merge:0' shape=(?, 128) dtype=float32>]


## Definitions

In [4]:
def extract_face(filename, required_size=(160, 160)):
    image = Image.open(filename)
    image = image.convert('RGB')
    pixels = asarray(image)
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

In [5]:
def load_faces(directory):
    faces = list()
    for filename in listdir(directory):
        path = directory + filename
        face = extract_face(path)
        faces.append(face)
    return faces

In [6]:
def load_dataset(directory):
    X, y = list(), list()
    for subdir in listdir(directory):
        path = directory + subdir + '/'
        faces = load_faces(path)
        labels = [subdir for _ in range(len(faces))]
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

## Dataset

In [7]:
train_X, train_y = load_dataset('data/train/')
test_X, test_y = load_dataset('data/val/')
savez_compressed('5-celebrity-faces-dataset.npz', 
                  train_X, 
                  train_y,
                  test_X,
                  test_y)

>loaded 0 examples for class: .ipynb_checkpoints

>loaded 14 examples for class: ben_afflek
>loaded 21 examples for class: jerry_seinfeld
>loaded 19 examples for class: madonna
>loaded 17 examples for class: elton_john
>loaded 22 examples for class: mindy_kaling
>loaded 0 examples for class: .ipynb_checkpoints
>loaded 5 examples for class: ben_afflek
>loaded 5 examples for class: jerry_seinfeld
>loaded 5 examples for class: madonna
>loaded 5 examples for class: elton_john
>loaded 5 examples for class: mindy_kaling


## Face embeddings

In [8]:
def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

In [9]:
data = load('5-celebrity-faces-dataset.npz')
train_X, train_y, test_X, test_y = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
model = load_model('Serialized/facenet_keras.h5')

In [12]:
embedded_train_X = list()
for face_pixels in train_X:
    embedding = get_embedding(model, face_pixels)
    embedded_train_X.append(embedding)
embedded_train_X = asarray(embedded_train_X)

In [13]:
embedded_test_X = list()
for face_pixels in test_X:
    embedding = get_embedding(model, face_pixels)
    embedded_test_X.append(embedding)
embedded_test_X = asarray(embedded_test_X)

In [14]:
savez_compressed('5-celebrity-faces-embeddings.npz',
                embedded_train_X,
                train_y,
                embedded_test_X,
                test_y)

## SVM classifier

In [15]:
data = load('5-celebrity-faces-embeddings.npz')
train_X, train_y, test_X, test_y = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']

In [16]:
in_encoder = Normalizer(norm='l2')
train_X = in_encoder.transform(train_X)
test_X = in_encoder.transform(test_X)

In [17]:
out_encoder = LabelEncoder()
out_encoder.fit(train_y)
train_y = out_encoder.transform(train_y)
test_y = out_encoder.transform(test_y)

In [20]:
model = SVC(kernel='linear', probability=True)
model.fit(train_X, train_y)
filename = 'svm.sav'
pickle.dump(model, open(filename, 'wb'))

In [21]:
yhat_train = model.predict(train_X)
yhat_test = model.predict(test_X)
score_train = accuracy_score(train_y, yhat_train)
score_test = accuracy_score(test_y, yhat_test)
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Accuracy: train=100.000, test=100.000
